In [2]:
import pandas as pd
# Cargar el archivo JSON en un DataFrame
df = pd.read_json('../data/ejercicios_integracion/competencia_kaggle.json')

print(df)

   id_participante          nombre  intentos  tiempo_total_horas  \
0                1  Participante_A         5                12.0   
1                2  Participante_B         3                 NaN   
2                3  Participante_C         4                10.0   
3                4  Participante_D         2                 5.0   
4                5  Participante_E         6                14.0   

                   puntuaciones  
0        [80, 85, 90, 88, None]  
1                [70, 75, None]  
2              [85, 90, 95, 93]  
3                    [60, None]  
4  [78, 82, None, None, 80, 85]  


In [ ]:
#Cargar valores con 0
df = df.fillna(0)

print(df)

# Obtener información general sobre el DataFrame
print("Información del DataFrame:")
print(df.info())
# Generar estadísticas descriptivas
print("Estadísticas descriptivas del DataFrame:")
print(df.describe())

   id_participante          nombre  intentos  tiempo_total_horas  \
0                1  Participante_A         5                12.0   
1                2  Participante_B         3                 0.0   
2                3  Participante_C         4                10.0   
3                4  Participante_D         2                 5.0   
4                5  Participante_E         6                14.0   

                   puntuaciones  
0        [80, 85, 90, 88, None]  
1                [70, 75, None]  
2              [85, 90, 95, 93]  
3                    [60, None]  
4  [78, 82, None, None, 80, 85]  
Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id_participante     5 non-null      int64  
 1   nombre              5 non-null      object 
 2   intentos            5 non-null      int64  
 3   tiempo_tot

In [6]:
# Calcular promedio, máximo y mínimo de 'intentos'
promedio_cantidad = df['intentos'].mean()
max_cantidad = df['intentos'].max()
min_cantidad = df['intentos'].min()

print(f"Promedio de intentos: {promedio_cantidad}")
print(f"Máximo de intentos: {max_cantidad}")
print(f"Mínimo de intentos: {min_cantidad}")

Promedio de intentos: 4.0
Máximo de intentos: 6
Mínimo de intentos: 2
